In [1]:
import numpy as np # linear algebra
import pandas as pd
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
# tqdm.pandas()

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib


import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, TimeDistributed, LeakyReLU, Conv1D, BatchNormalization, MaxPooling1D, AveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from xgboost import XGBClassifier, plot_importance

from Model_class import CNN_model, CNN_Tree_Classifier, CNN_Bagging, CNN_Boosting
from Model_function import Hyperparameter_Tuning as tuning

import pickle
import random

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
cluster = 0
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\Cluster_{cluster}_classification_minmax0_Weekly', 'rb') as fp:
    load_list = pickle.load(fp)
len(load_list)

6

In [3]:
param = {'Conv_layer': [2, 3, 4],
        'Dense_layer': [1, 2],
        'Dense': [64, 128, 256],
        'dropout_ratio': [0, 0.1, 0.15, 0.2, 0.25, 0.3],
        'filter': [64, 100, 128, 256],
        'kernel': [3, 5],
        'padding': ['causal', 'same'],
        'learning_rate': [0.001, 0.002, 0.003],
        'batch': [64, 128, 256, 512],
        'epochs': [1]}

p = {'classifier': ['RandomForest'],
    'max_depth': [5, 10, 15, 20, 25, 30],
    'max_features': ['auto'],
    'min_samples_leaf': [0.05, 0.1, 0.15, 0.2],
    'min_samples_split': [0.05, 0.1, 0.15, 0.2],
    'n_estimators': [10,20]}

p_a = {'n_estimator': [1,2],
                                'subsample': [0.8]}

In [4]:
Optimization = tuning(load_list, model='CNN_Boosting', CNNparam=param, Modelparam=p_a, max_iter=1, cluster_num=0, model_keep=1)

In [5]:
Optimization._Modelparam

{'n_estimator': [1, 2], 'subsample': [0.8]}

In [6]:
Optimization.RandomSearch()

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and

W1023 13:28:47.166129  2228 deprecation.py:506] From C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 197067 samples, validate on 27592 samples
197067/197067 - 5s - loss: 0.6781 - acc: 0.5768 - val_loss: 0.6766 - val_acc: 0.5812
Train on 157653 samples, validate on 27592 samples
157653/157653 - 3s - loss: 0.6924 - acc: 0.5178 - val_loss: 0.6899 - val_acc: 0.5456



C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


AUC:	0.5789997581410469 	 Accuracy:	0.5811829515801682 	 Accuracy Balance:	0.6575342465753424 	 AUC Balance:	0.7671232876712328


In [9]:
Optimization.Overall_Evaluation(1)


(0.2153846153846154, 0.26153846153846155)

In [10]:
Optimization.load_optimal_model()

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and

In [17]:
Optimization.optimal_CNN

[{'Conv_layer': 3,
  'Dense_layer': 2,
  'Dense': [64, 128],
  'dropout_ratio': [0.25, 0.3, 0.15, 0.3, 0],
  'filter': [100, 64, 100],
  'kernel': [5, 3, 5],
  'padding': ['same', 'same', 'same'],
  'learning_rate': 0.001,
  'batch': 64,
  'epochs': 1}]

In [10]:
assert 'test' in ['test', 't']

In [14]:
assert len({1:0, 2:0, 3:0}) < 4